## Miwok Entrants Database

### This notebook reads in the entrants table from miwok100k website and cross references ultrasignup results. 

### The data is saved to csv and read into Google Sheets here:

#### https://docs.google.com/spreadsheets/d/1axjigyHEji-qD9gAyqzKU7AbUOkW0rrKDPUHJW-sI68/edit?usp=sharing

In [1]:
from bs4 import BeautifulSoup
import urllib
import pandas as pd
import json
from IPython.display import HTML
import numpy as np

response = urllib.request.urlopen('http://miwok100k.com/site/lottery/')
html = response.read()
soup = BeautifulSoup(html)

entrant_table = soup.find("table", {"class" : "tableizer-table"})

# get the rows in the table (excluding header)
foo = entrant_table.findAll('tr')
header = foo[0]
entrants = foo[1:]



In [2]:
records = [] # store all of the records in this list
for row in entrants:
    col = row.findAll('td')
    #orig_index = 
    firstname = col[0].string.strip()
    lastname = col[1].string.strip()
    record = '%s,%s' % (firstname, lastname) # store the record, comma separated
    records.append(record)

In [3]:
records

['Will,Aarsheim',
 'Shahid,Ali',
 'Kay,Allen',
 'Erik,Andersen',
 'John,Anderson',
 'Nikolay,Anikevich',
 'Bobb,Ankeney',
 'Megan,Arauzo',
 'Jefferson,Arbuckle',
 'Jeanie,Arnold',
 'Chris,Aversa',
 'Angel,Avila',
 'Tom,Bache',
 'William,Bacon',
 'Audrey,Baldessari',
 'Nicholas,Banaszak',
 'Kelly,Barber',
 'Denise,Barchas',
 'Noe,Barrios',
 'jason,bates',
 'Keith,Bauer',
 'Leah,Beck',
 'Lisa,Belmonte',
 'Rogelio,Bennett',
 'Daniel,Berger',
 'Troy,Bertram',
 'Jessica,Bird',
 'Rhea,Black',
 'Andy,Black',
 'Chris,Blanchard',
 'Ilene,Bloom',
 'Kate,Bobovski',
 'Mike,Bohi',
 'Dennis,Boic',
 'Timothy,Bolen',
 'Lance,Brady',
 'Jody,Braninburg',
 'Gregory,Brant',
 'Jonathan,Bretan',
 'Daniel,Brostella',
 'Thomas,Brown',
 'Patrick,Brown',
 'Gretchen,Brugman',
 'Turk,Brunsman',
 'Moriah,Buckley',
 'Jason,Buckman',
 'alison,burke',
 'Katie,Burns',
 'Charles,Bybee',
 'Tim,Callahan',
 'Coral,Candlish-Rutherford',
 'Pete,Cannon',
 'Cristian,Carata',
 'Adam,Cardenas',
 'Rodrigo,Cardoze',
 'Rebecca,Car

In [4]:


#split by comma and store in dataframe
records = [r.split(',') for r in records]

entrants = pd.DataFrame(records)
entrants.columns = ['First','Last']
# make first and last names have %20 instead of spaces
entrants['First'] = [r.replace(" ", "%20") for r in entrants['First']]
entrants['Last'] = [r.replace(" ", "%20") for r in entrants['Last']]

# initialize gender, etc. columns
entrants['Gender'] = ''
entrants['AgeRank'] = 0
entrants['Rank'] = 0
entrants['Age'] = 0
entrants['NumRaces'] = 0
entrants['LinkHTML'] = ''
entrants['LinkURL'] = ''
entrants['NumMatches'] = 0
entrants['Selected'] = 0 #index of runner selected (useful for de-duping)
#entrants.head()

In [5]:
entrants.head()

,First,Last,Gender,AgeRank,Rank,Age,NumRaces,LinkHTML,LinkURL,NumMatches,Selected
0,Will,Aarsheim,,0,0,0,0,,,0,0
1,Shahid,Ali,,0,0,0,0,,,0,0
2,Kay,Allen,,0,0,0,0,,,0,0
3,Erik,Andersen,,0,0,0,0,,,0,0
4,John,Anderson,,0,0,0,0,,,0,0


In [6]:
# This cell takes some time, it cross references the ultrasignup data
no_match = []
for i in range(len(entrants)):
    f = entrants.First[i]
    l = entrants.Last[i]
    try: 
        l.encode('ascii')
    except:
        print('unicode error')
        no_match.append(i)
        entrants.loc[i,['Gender']] =  'None'
        entrants.loc[i,['Rank']] =  'None'
        entrants.loc[i,['AgeRank']] =  'None'
        continue
    try:        
        r = urllib.request.urlopen('https://ultrasignup.com/service/events.svc/history/' + f + '/' + l + '/')
        u = 'https://ultrasignup.com/results_participant.aspx?fname=' + f +'&lname=' + l
        u2 = '<a href="{0}">link</a>'.format(u)
        
        data = json.loads(r.readall().decode('utf-8'))
        entrants.loc[i,['NumMatches']] = len(data) #for resolving duplicate records
        
        # if num matches is >1, choose the runner with the larger # results
        if len(data) > 1:
            # make a list of the num races 
            races = [len(data[j]['Results']) for j in range(len(data))]
            #select index corresponding to largest races number
            ind = races.index(max(races))
        else:
            ind = 0 #there is only one runner, corresponding to index 0 in data
        
        entrants.loc[i,['Gender']] =  data[ind]['Gender']
        entrants.loc[i,['Rank']] =  data[ind]['Rank']
        entrants.loc[i,['AgeRank']] =  data[ind]['AgeRank']
        entrants.loc[i,['NumRaces']]= len(data[ind]['Results'])
        entrants.loc[i,['Age']]= data[ind]['Age']
        entrants.loc[i,['LinkHTML']]= u2 
        entrants.loc[i,['LinkURL']]= u
        entrants.loc[i,['Selected']] = ind #keep track of which runner is chosen for de-duping
        
    except (IndexError, urllib.request.HTTPError):
        no_match.append(i)
        entrants.loc[i,['Gender']] =  'None'
        entrants.loc[i,['Rank']] =  'None'
        entrants.loc[i,['AgeRank']] =  'None'
print('there were ', len(no_match), 'entrants with no match in ultrasignup')

unicode error
unicode error
unicode error
unicode error
unicode error
unicode error
unicode error
there were  25 entrants with no match in ultrasignup


In [7]:
entrants = entrants.replace('None', np.nan)

In [8]:
entrants[['Rank']] = entrants[['Rank']].astype(float)

In [9]:
entrants.sort_values(by='Rank', ascending=False)

,First,Last,Gender,AgeRank,Rank,Age,NumRaces,LinkHTML,LinkURL,NumMatches,Selected
26,Jessica,Bird,F,1.0000,1.0000,34,1,"<a href=""https://ultrasignup.com/results_parti...",https://ultrasignup.com/results_participant.as...,1,0
108,Serena,Eley,F,0.9689,0.9640,37,11,"<a href=""https://ultrasignup.com/results_parti...",https://ultrasignup.com/results_participant.as...,1,0
269,Justus,Meyer,M,0.9555,0.9555,36,11,"<a href=""https://ultrasignup.com/results_parti...",https://ultrasignup.com/results_participant.as...,1,0
322,Ragan,Petrie,F,0.9639,0.9398,52,84,"<a href=""https://ultrasignup.com/results_parti...",https://ultrasignup.com/results_participant.as...,1,0
167,Leah,Handelman,F,0.9537,0.9332,32,11,"<a href=""https://ultrasignup.com/results_parti...",https://ultrasignup.com/results_participant.as...,1,0
119,Sarah,Ferguson,F,0.9623,0.9329,31,14,"<a href=""https://ultrasignup.com/results_parti...",https://ultrasignup.com/results_participant.as...,4,0
27,Rhea,Black,F,1.0000,0.9292,46,13,"<a href=""https://ultrasignup.com/results_parti...",https://ultrasignup.com/results_participant.as...,1,0
238,Brett,Long,M,0.9664,0.9282,40,30,"<a href=""https://ultrasignup.com/results_parti...",https://ultrasignup.com/results_participant.as...,3,0
434,Sarah,Weigel,F,0.9489,0.9272,41,23,"<a href=""https://ultrasignup.com/results_parti...",https://ultrasignup.com/results_participant.as...,1,0
438,Karen,Welwood,F,0.9893,0.9217,45,5,"<a href=""https://ultrasignup.com/results_parti...",https://ultrasignup.com/results_participant.as...,1,0


## Now Rank the Women According to UltraSignup "Rank"

In [10]:
pd.set_option('display.max_colwidth', -1)
#make separate female data frame :
F = entrants[(entrants['Gender']=='F')].sort_values(by='Rank', \
        ascending=0)
# include 'female rank' column and set it as index
F['FRank'] = list(range(len(F)))
F['FRank'] = F['FRank'] + 1
F = F.set_index('FRank')
# list all the females, sort by rank, display as HTML so that link works
HTML(F[['First','Last','Rank','Age','NumRaces','LinkHTML']].head(30).to_html(escape=False))

/Users/stephanie/anaconda3/lib/python3.4/site-packages/pandas/core/computation/check.py:17: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


,First,Last,Rank,Age,NumRaces,LinkHTML
FRank,,,,,,
1,Jessica,Bird,1.0000,34,1,link
2,Serena,Eley,0.9640,37,11,link
3,Ragan,Petrie,0.9398,52,84,link
4,Leah,Handelman,0.9332,32,11,link
5,Sarah,Ferguson,0.9329,31,14,link
6,Rhea,Black,0.9292,46,13,link
7,Sarah,Weigel,0.9272,41,23,link
8,Karen,Welwood,0.9217,45,5,link
9,Gwen,Golaszewski,0.9210,30,9,link


## Rank the Men, too.

In [12]:
#make separate male data frame :
M = entrants[(entrants['Gender']=='M')].sort_values(by='Rank', \
        ascending=0)
# include 'male rank' column and set it as index
M['MRank'] = list(range(len(M)))
M['MRank'] = M['MRank'] + 1
M = M.set_index('MRank')
# list all the females, sort by rank, display as HTML so that link works
HTML(M[['First','Last','Rank','Age','NumRaces','LinkHTML']].head(50).to_html(escape=False))

,First,Last,Rank,Age,NumRaces,LinkHTML
MRank,,,,,,
1,Justus,Meyer,0.9555,36,11,link
2,Brett,Long,0.9282,40,30,link
3,Paul,Terranova,0.9138,44,87,link
4,Mark,Thompson,0.9115,40,32,link
5,Kevin,Sullivan,0.9096,42,15,link
6,Jean,Pommier,0.8982,54,149,link
7,Franz,van%20der%20Groen,0.8979,33,32,link
8,Adam,Casseday,0.8958,37,51,link
9,Brian,Miller,0.8890,42,63,link


In [13]:
# get rid of the %20's in the names now
entrants['First'] = [r.replace("%20", " ") for r in entrants['First']]
entrants['Last'] = [r.replace("%20", " ") for r in entrants['Last']]

In [14]:
entrants.to_csv('Miwok2018Entrants.csv', encoding='utf-8')
